In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install transformers
!pip install simpletransformers

In [ ]:
# install simpleT5
!pip install simplet5

In [ ]:
import numpy as np
import pandas as pd
import os, json, gc, re, random
from tqdm.notebook import tqdm
# from tqdm import tqdm
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch, transformers, tokenizers
torch.__version__, transformers.__version__, tokenizers.__version__

('1.8.1+cu101', '4.6.1', '0.10.3')

In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


##MOH-X - Abstract Summarization (Noun+Verb)

In [ ]:
data_file = pd.read_csv("data/trofi.csv")

In [ ]:
data_file = data_file.rename(columns={'sentence': 'source_text'}) #abstract - sentence
data_file['target_text'] = data_file['verb'] #title - metaphorical words
#data_file['target_text'] = data_file['verb'] #title - metaphorical words

In [ ]:
papers = data_file[["source_text", "target_text"]]
papers.head()

,source_text,target_text
0,An Energy Department spokesman says the sulfur...,absorb
1,The yellow beta carotene pigment absorbs blue ...,absorb
2,"This time , the ground absorbed the shock wave...",absorb
3,'' Vitamins could be passed right out of the b...,absorb
4,"As Eliot wrote : '' In a warm haze , the sultr...",absorb


In [ ]:
# split the data into training and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(papers, test_size=0.1)

In [ ]:
# import
from simplet5 import SimpleT5

# instatntiate
model = SimpleT5()

# load
model.from_pretrained("t5","t5-base")

# train
model.train(train_df=train_df, eval_df=test_df, source_max_token_len=512, target_max_token_len=128, max_epochs=5, batch_size=8, use_gpu=True)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.core.lightning:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!ls outputs/

config.json			    SimpleT5-epoch-2-train-loss-0.2695
eval_results.txt		    SimpleT5-epoch-3-train-loss-0.0493
merges.txt			    SimpleT5-epoch-3-train-loss-0.1619
model_args.json			    SimpleT5-epoch-3-train-loss-0.1764
pytorch_model.bin		    SimpleT5-epoch-4-train-loss-0.037
SimpleT5-epoch-0-train-loss-0.8219  SimpleT5-epoch-4-train-loss-0.1171
SimpleT5-epoch-0-train-loss-1.4472  SimpleT5-epoch-4-train-loss-0.1437
SimpleT5-epoch-0-train-loss-1.915   special_tokens_map.json
SimpleT5-epoch-1-train-loss-0.0803  tokenizer_config.json
SimpleT5-epoch-1-train-loss-0.4197  tokenizer.json
SimpleT5-epoch-1-train-loss-0.5195  training_args.bin
SimpleT5-epoch-2-train-loss-0.05    vocab.json
SimpleT5-epoch-2-train-loss-0.2453


In [ ]:
# load a trained model
model.load_model("outputs/SimpleT5-epoch-4-train-loss-1.1577", use_gpu=True)

In [ ]:
# let's see how it performerd:
sample_abstracts = test_df.sample(65)

for i, abstract in sample_abstracts.iterrows():
    print(f"===== Original Sentence =====")
    print(abstract['source_text'])
    summary= model.predict(abstract['source_text'])[0]
    print(f"\n===== Actual Metaphorical Tokens =====")
    print(f"{abstract['target_text']}")
    print(f"\n===== Predicted Metaphorical Tokens =====")
    print(f"{summary}")
    print("\n +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

===== Original Sentence =====
Mr. Ketelsen said yesterday that said their interest is greater than the company expected , but that '' we may have people kicking tires . '

===== Actual Metaphorical Tokens =====
kick

===== Predicted Metaphorical Tokens =====
kick

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
Periodically , a nurse emerges from one of eight examining rooms to bellow out the name of the next patient 

===== Actual Metaphorical Tokens =====
examine

===== Predicted Metaphorical Tokens =====
examine

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

===== Original Sentence =====
Luby 's Cafeterias Inc . , San Antonio , Texas , is targeting the Midwest after recently inching out of its Deep South stronghold into Arizona 

===== Actual Metaphorical Tokens =====
target

===== Predicted Metaphorical Tokens =====
target

 +++++++++++++++++++++++++++++++++++++++++++++++++++++++